In [1]:
!pip install nltk

In [3]:
!pip install textblob

   ---------------------------------------- 0.0/624.3 kB ? eta -:--:--
   ---------------------------------------- 624.3/624.3 kB 3.9 MB/s eta 0:00:00


In [6]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from datetime import datetime

In [48]:
df = pd.read_csv('C:\\Users\\Admin\\excel\\top10K-TMDB-movies.csv')

In [50]:
df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [52]:
df.tail()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
9995,10196,The Last Airbender,"Action,Adventure,Fantasy",en,"The story follows the adventures of Aang, a yo...",98.322,2010-06-30,4.7,3347
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",en,The sharks take bite out of the East Coast whe...,12.490,2015-07-22,4.7,417
9997,13995,Captain America,"Action,Science Fiction,War",en,"During World War II, a brave, patriotic Americ...",18.333,1990-12-14,4.6,332
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",en,A man named Farmer sets out to rescue his kidn...,15.159,2007-11-29,4.7,668
9999,455957,Domino,"Thriller,Action,Crime",en,Seeking justice for his partner’s murder by an...,16.482,2019-05-31,4.6,221


In [54]:
df.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [56]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.3+ KB


In [58]:
df.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [60]:
df.isna().sum()

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [62]:
df['overview'].fillna('No overview available', inplace=True)
df['genre'].fillna('Unknown genre', inplace=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_11496\3312236932.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['overview'].fillna('No overview available', inplace=True)
C:\Users\Admin\AppData\Local\Temp\ipykernel_11496\3312236932.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a co

In [64]:
df.isna().sum()

id                   0
title                0
genre                0
original_language    0
overview             0
popularity           0
release_date         0
vote_average         0
vote_count           0
dtype: int64

In [66]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [68]:
import nltk
print(nltk.data.path)


['C:\\Users\\Admin/nltk_data', 'C:\\Users\\Admin\\anaconda3\\nltk_data', 'C:\\Users\\Admin\\anaconda3\\share\\nltk_data', 'C:\\Users\\Admin\\anaconda3\\lib\\nltk_data', 'C:\\Users\\Admin\\AppData\\Roaming\\nltk_data', 'C:\\nltk_data', 'D:\\nltk_data', 'E:\\nltk_data']


In [70]:
nltk.data.path.append('C:/Users/Admin/nltk_data')

In [131]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\abc.zip.
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping corpora\alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers\averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       tagge

True

In [72]:
#converting data into lower case
df['overview'] = df['overview'].str.lower()

In [74]:
#initializing stopwords
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [76]:
def preprocess_text(text):
    if isinstance(text, str):  # Check if the value is a string
        tokens = word_tokenize(text)
        filtered_tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and word.isalnum()]
        return ' '.join(filtered_tokens)
    return np.nan  # Return NaN for non-string values

# Apply preprocessing safely
df['overview'] = df['overview'].apply(preprocess_text)


In [78]:
encoder = OneHotEncoder()
genre_encoded = encoder.fit_transform(df[['genre']]).toarray()
language_encoded = encoder.fit_transform(df[['original_language']]).toarray()

In [80]:
# Import necessary library
from sklearn.preprocessing import OneHotEncoder

In [82]:
# Perform one-hot encoding for 'genre'
encoder = OneHotEncoder(sparse_output=False)
genre_encoded = encoder.fit_transform(df[['genre']])  # Perform encoding

In [84]:
# Correctly generate column names
genre_df = pd.DataFrame(genre_encoded, columns=encoder.get_feature_names_out(['genre']))

In [86]:
# Similarly, for 'original_language'
language_encoded = encoder.fit_transform(df[['original_language']])
language_df = pd.DataFrame(language_encoded, columns=encoder.get_feature_names_out(['original_language']))

In [88]:
# Concatenate the encoded DataFrames back to the main DataFrame
df = pd.concat([df, genre_df, language_df], axis=1)
print(df.shape)

(10000, 2176)


In [90]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df['release_year'] = df['release_date'].dt.year

In [92]:
scaler = MinMaxScaler()
numerical_columns = ['popularity', 'vote_average', 'vote_count']
df[numerical_columns] = scaler.fit_transform(df[numerical_columns])

In [94]:
df.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count,genre_Action,...,original_language_sh,original_language_sr,original_language_sv,original_language_te,original_language_th,original_language_tr,original_language_uk,original_language_xx,original_language_zh,release_year
0,278,The Shawshank Redemption,"Drama,Crime",en,framed 1940s double murder wife lover upstandi...,0.008957,1994-09-23,1.00000,0.682978,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1994
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,raj rich carefree second generation nri simran...,0.002377,1995-10-19,1.00000,0.111328,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1995
2,238,The Godfather,"Drama,Crime",en,spanning year 1945 1955 chronicle fictional co...,0.008622,1972-03-14,1.00000,0.506984,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1972
3,424,Schindler's List,"Drama,History,War",en,true story businessman oskar schindler saved t...,0.004231,1993-12-15,0.97561,0.402276,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1993
4,240,The Godfather: Part II,"Drama,Crime",en,continuing saga corleone crime family young vi...,0.005476,1974-12-20,0.97561,0.303024,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1974


In [96]:
df.tail()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count,genre_Action,...,original_language_sh,original_language_sr,original_language_sv,original_language_te,original_language_th,original_language_tr,original_language_uk,original_language_xx,original_language_zh,release_year
9995,10196,The Last Airbender,"Action,Adventure,Fantasy",en,story follows adventure aang young successor l...,0.009364,2010-06-30,0.02439,0.099221,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2010
9996,331446,Sharknado 3: Oh Hell No!,"Action,TV Movie,Science Fiction,Comedy,Adventure",en,shark take bite east coast sharknado hit washi...,0.001139,2015-07-22,0.02439,0.006842,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2015
9997,13995,Captain America,"Action,Science Fiction,War",en,world war ii brave patriotic american soldier ...,0.001699,1990-12-14,0.00000,0.004162,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1990
9998,2312,In the Name of the King: A Dungeon Siege Tale,"Adventure,Fantasy,Action,Drama",en,man named farmer set rescue kidnapped wife ave...,0.001395,2007-11-29,0.02439,0.014755,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2007
9999,455957,Domino,"Thriller,Action,Crime",en,seeking justice partner murder isi member cope...,0.001522,2019-05-31,0.00000,0.000662,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2019
